In [62]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [63]:
yelp = pd.read_csv('Yelp.csv')

yelp['rating'] = 0

mask= yelp['stars']>3

yelp['rating'][mask] = 1


/Users/leeanthea/anaconda/envs/dato-env/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [64]:
from sklearn.cross_validation import StratifiedShuffleSplit 

thousand = yelp[:1000]

y = thousand['rating']

X = thousand.drop(['stars','Review','rating'], axis=1)

Review = thousand['Review']

### we need a test set that we didn't train on to find the best weights for combining the classifiers
sss = StratifiedShuffleSplit(y, test_size=0.3, random_state=1234)
for train_index, test_index in sss:
    break

train_x, train_y,train_review = X.values[train_index],y.values[train_index],Review[train_index]
test_x, test_y, test_review = X.values[test_index], y.values[test_index],Review[test_index]

In [75]:
import nltk
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


import re
import string
from nltk import word_tokenize          
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords


stemmer = PorterStemmer()
wnl = WordNetLemmatizer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def lemmatize(tokens,wnl):
    lem=[]
    for item in tokens:
        lem.append(wnl.lemmatize(item))
    return lem

def my_tokenizer(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    text = re.sub('[0-9]+','',text)
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    #result = stem_tokens(tokens, stemmer)
    #result = lemmatize(tokens, wnl)
    result = tokens
    return result
    

vectorizer = TfidfVectorizer(min_df=3,max_df=0.9,tokenizer=my_tokenizer,
 ngram_range=(1, 2), 
 stop_words='english',
 strip_accents='unicode',max_features=200)

text_1 = vectorizer.fit_transform(train_review)

In [76]:
from nltk import NaiveBayesClassifier

from sklearn import metrics

text_2 = vectorizer.transform(test_review)

from sklearn.naive_bayes import MultinomialNB

nb =  MultinomialNB().fit(text_1, train_y)

y_nb_predicted = nb.predict(text_2)


print "MODEL: Multinomial Naive Bayes\n"

print 'The precision for this classifier is ' + str(metrics.precision_score(test_y, y_nb_predicted))
print 'The recall for this classifier is ' + str(metrics.recall_score(test_y, y_nb_predicted))
print 'The f1 for this classifier is ' + str(metrics.f1_score(test_y, y_nb_predicted))
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, y_nb_predicted))

MODEL: Multinomial Naive Bayes

The precision for this classifier is 0.740484429066
The recall for this classifier is 1.0
The f1 for this classifier is 0.850894632207
The accuracy for this classifier is 0.75


In [77]:
from sklearn.naive_bayes import MultinomialNB
from operator import itemgetter

N = 50
vocabulary = np.array([t for t, i in sorted(vectorizer.vocabulary_.iteritems(), key=itemgetter(1))])

topN = np.argsort(nb.coef_[0])[-N:]

print "\nThe top %d most informative features: \n%s" % (N, ','.join(vocabulary[topN]))


The top 50 most informative features: 
spicy,sauce,patio,meal,tiki,eat,phoenix,potato,shrimp,order,dinner,pork,amazing,tasty,fries,excellent,atmosphere,friendly,ordered,rice,went,favorite,definitely,lunch,awesome,fish,happy hour,best,nice,restaurant,drinks,menu,little,sweet,got,hour,try,happy,delicious,love,time,chicken,service,really,hula,like,good,place,food,great


In [78]:
most_informative = vocabulary[topN]

for w in most_informative:
    X[w]= 0

def create_features(tokens,i):
    document_words = set(tokens)
    features = {}
    for word in most_informative:
        if word in document_words:
            X.loc[i,word] = 1
        #features['contains({})'.format(word)] = (word in document_words)
    #return features   

In [79]:
X.head()

,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,French,...,time,chicken,service,really,hula,like,good,place,food,great
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,9,9,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
for i in range(1000):
    text = "".join([ch for ch in Review[i] if ch not in string.punctuation])
    text = re.sub('[0-9]+','',text)
    tokens = nltk.word_tokenize(text.lower().decode('utf-8'))
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    feature = create_features(tokens,i)

In [81]:
X.head()

,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,French,...,time,chicken,service,really,hula,like,good,place,food,great
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,2,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
2,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,0,1,2,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,1,0
4,7,9,9,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [82]:
nb_text =  MultinomialNB().fit(X.values[train_index], train_y)

text_predicted = nb_text.predict(X.values[test_index])


print "MODEL: Multinomial Naive Bayes\n"

print 'The precision for this classifier is ' + str(metrics.precision_score(test_y, text_predicted))
print 'The recall for this classifier is ' + str(metrics.recall_score(test_y, text_predicted))
print 'The f1 for this classifier is ' + str(metrics.f1_score(test_y, text_predicted))
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, text_predicted))

MODEL: Multinomial Naive Bayes

The precision for this classifier is 0.775933609959
The recall for this classifier is 0.873831775701
The f1 for this classifier is 0.821978021978
The accuracy for this classifier is 0.73


In [95]:
sample_weight = np.random.rand(y.shape[0])

sw_train, sw_test = sample_weight[train_index], sample_weight[test_index]

from sklearn.calibration import CalibratedClassifierCV

from sklearn.metrics import brier_score_loss


clf = MultinomialNB()
model_original = clf.fit(X.values[train_index], train_y)  # GaussianNB itself does not support sample-weights
prob_pos_clf = clf.predict_proba(X.values[test_index])[:, 1]
clf_predicted =model_original.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, clf_predicted))

# Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(clf, cv=10, method='isotonic')
model_isotonic = clf_isotonic.fit(X.values[train_index], train_y, sw_train)
prob_pos_isotonic = clf_isotonic.predict_proba(X.values[test_index])[:, 1]
isotonic_predicted = model_isotonic.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, isotonic_predicted))

# Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(clf, cv=10, method='sigmoid')
model_sigmoid  = clf_sigmoid.fit(X.values[train_index], train_y, sw_train)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X.values[test_index])[:, 1]
sigmoid_predicted = model_sigmoid.predict(X.values[test_index])
print 'The accuracy for this classifier is ' + str(metrics.accuracy_score(test_y, sigmoid_predicted))

print("Brier scores: (the smaller the better)")

clf_score = brier_score_loss(test_y, prob_pos_clf, sw_test)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(test_y, prob_pos_isotonic, sw_test)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(test_y, prob_pos_sigmoid, sw_test)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

The accuracy for this classifier is 0.73
The accuracy for this classifier is 0.76
The accuracy for this classifier is 0.746666666667
Brier scores: (the smaller the better)
No calibration: 0.172
With isotonic calibration: 0.169
With sigmoid calibration: 0.168
